In [1]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

import pandas as pd

# Display the complete contents of dataframe cells.
pd.set_option("display.max_colwidth", None)

In [2]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

from phoenix.otel import register

tracer_provider = register(endpoint="http://0.0.0.0:6006/v1/traces")
LlamaIndexInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://0.0.0.0:6006/v1/traces
|  Transport: HTTP
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



### Load the test queries

In [3]:
# Load the DataFrame from the CSV file
test_df = pd.read_csv("../data/evaluation/test_queries.csv", index_col=0)

# Display the first few rows of the loaded DataFrame
test_df.head()

,query
0,Who is the president?
1,Who scored the most goals in the European Champion's League in 2020?
2,articles similar to the article on 'Philosophy'?
3,the school of athens
4,css flex


### Build the Agent

In [5]:
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import ToolSelection, ToolOutput
from llama_index.core.workflow import Event
class PrepEvent(Event):
    pass


class InputEvent(Event):
    input: list[ChatMessage]


class ToolCallEvent(Event):
    tool_calls: list[ToolSelection]


class FunctionOutputEvent(Event):
    output: ToolOutput

In [6]:
from typing import Any, List, Union

from llama_index.core.agent.react import ReActChatFormatter, ReActOutputParser
from llama_index.core.agent.react.types import (
    ActionReasoningStep,
    ObservationReasoningStep,
)
from llama_index.core.llms.llm import LLM
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.tools.types import BaseTool
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    step,
)
from llama_index.llms.openai import OpenAI


class ReActAgent(Workflow):
    def __init__(
        self,
        *args: Any,
        llm: Union[LLM, None] = None,
        tools: Union[list[BaseTool], None] = None,
        extra_context: Union[str, None] = None,
        **kwargs: Any,
    ) -> None:
        super().__init__(*args, **kwargs)
        self.tools = tools or []

        self.llm = llm or OpenAI()

        self.memory = ChatMemoryBuffer.from_defaults(llm=llm)
        self.formatter = ReActChatFormatter(context=extra_context or "")
        self.output_parser = ReActOutputParser()
        self.sources = []

    @step
    async def new_user_msg(self, ctx: Context, ev: StartEvent) -> PrepEvent:
        # clear sources
        self.sources = []

        # get user input
        user_input = ev.input
        user_msg = ChatMessage(role="user", content=user_input)
        self.memory.put(user_msg)

        # clear current reasoning
        await ctx.set("current_reasoning", [])

        return PrepEvent()

    @step
    async def prepare_chat_history(
        self, ctx: Context, ev: PrepEvent
    ) -> InputEvent:
        # get chat history
        chat_history = self.memory.get()
        current_reasoning = await ctx.get("current_reasoning", default=[])
        llm_input = self.formatter.format(
            self.tools, chat_history, current_reasoning=current_reasoning
        )
        return InputEvent(input=llm_input)

    @step
    async def handle_llm_input(
        self, ctx: Context, ev: InputEvent
    ) -> Union[ToolCallEvent, StopEvent, PrepEvent]:
        chat_history = ev.input

        response = await self.llm.achat(chat_history)

        try:
            reasoning_step = self.output_parser.parse(response.message.content)
            (await ctx.get("current_reasoning", default=[])).append(
                reasoning_step
            )
            if reasoning_step.is_done:
                self.memory.put(
                    ChatMessage(
                        role="assistant", content=reasoning_step.response
                    )
                )
                return StopEvent(
                    result={
                        "response": reasoning_step.response,
                        "sources": [*self.sources],
                        "reasoning": await ctx.get(
                            "current_reasoning", default=[]
                        ),
                    }
                )
            elif isinstance(reasoning_step, ActionReasoningStep):
                tool_name = reasoning_step.action
                tool_args = reasoning_step.action_input
                return ToolCallEvent(
                    tool_calls=[
                        ToolSelection(
                            tool_id="fake",
                            tool_name=tool_name,
                            tool_kwargs=tool_args,
                        )
                    ]
                )
        except Exception as e:
            (await ctx.get("current_reasoning", default=[])).append(
                ObservationReasoningStep(
                    observation=f"There was an error in parsing my reasoning: {e}"
                )
            )

        # if no tool calls or final response, iterate again
        return PrepEvent()

    @step
    async def handle_tool_calls(
        self, ctx: Context, ev: ToolCallEvent
    ) -> PrepEvent:
        tool_calls = ev.tool_calls
        tools_by_name = {tool.metadata.get_name(): tool for tool in self.tools}

        # call tools -- safely!
        for tool_call in tool_calls:
            tool = tools_by_name.get(tool_call.tool_name)
            if not tool:
                (await ctx.get("current_reasoning", default=[])).append(
                    ObservationReasoningStep(
                        observation=f"Tool {tool_call.tool_name} does not exist"
                    )
                )
                continue

            try:
                tool_output = tool(**tool_call.tool_kwargs)
                self.sources.append(tool_output)
                (await ctx.get("current_reasoning", default=[])).append(
                    ObservationReasoningStep(observation=tool_output.content)
                )
            except Exception as e:
                (await ctx.get("current_reasoning", default=[])).append(
                    ObservationReasoningStep(
                        observation=f"Error calling tool {tool.metadata.get_name()}: {e}"
                    )
                )

        # prep the next iteraiton
        return PrepEvent()

In [8]:
import wikipedia
from pydantic import BaseModel
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from typing import Dict

class WikiSearchResult(BaseModel):
    title: str
    url: str

class WikiArticle(BaseModel):
    title: str
    content: str
    url: str
    
def wikipedia_similar_articles(query: str) -> list[Dict[str,str]]:
    """Search Wikipedia for articles similar to the given query and return titles and URLs."""
    search_results = wikipedia.search(query, results=5)
    result_list = []
    for result in search_results:
        try:
            page = wikipedia.page(result)
            result_list.append(WikiSearchResult(title=page.title, url=page.url))
        except wikipedia.exceptions.DisambiguationError as e:
            # Handle disambiguation pages by logging or ignoring
            print(f"Disambiguation page: {e.options}")
        except wikipedia.exceptions.PageError:
            print(f"PageError: {result}")
    return result_list



def wikipedia_full_article(query: str) -> Dict[str,str]:
    """Fetch the full Wikipedia article for the given query."""
    try:
        page = wikipedia.page(query)
        return WikiArticle(title=page.title, content=page.content, url=page.url)
    except wikipedia.exceptions.DisambiguationError as e:
        # Handle disambiguation pages
        print(f"Disambiguation page: {e.options}")
    except wikipedia.exceptions.PageError:
        print(f"PageError: {query}")
    return None


# Wrap these functions in a tool
similar_articles_tool = FunctionTool.from_defaults(fn=wikipedia_similar_articles)
full_article_tool = FunctionTool.from_defaults(fn=wikipedia_full_article)



In [9]:
llm = OpenAI(model="gpt-4o")
agent = ReActAgent(
    llm=llm, tools=[similar_articles_tool, full_article_tool], timeout=120, verbose=True
)

In [10]:
from phoenix.trace.dsl.helpers import SpanQuery
import phoenix as px

client = px.Client()

In [11]:
import pandas as pd
from datasets import Dataset
from tqdm.auto import tqdm

from phoenix.trace import using_project

from typing import List, Union
# for normal python
# import asyncio
# for jupyter notebook
import nest_asyncio
nest_asyncio.apply()
agent.memory.reset()
from phoenix.trace import using_project

def get_context(response) -> List[str]:
    content = []  
    for source in response['sources']:
        tool_output = source.raw_output
        if isinstance(tool_output, list) and all(isinstance(item, WikiSearchResult) for item in tool_output):
            for result in tool_output:
                content.append(f"Title: {result.title}, URL: {result.url}")
        elif isinstance(tool_output, WikiArticle):
            content.append(f"Title: {tool_output.title}, Content: {tool_output.content}, URL: {tool_output.url}")
    return content

async def generate_response(agent, question) -> Dict[str, Union[str, List[str]]]:
    response = await agent.run(input=question)
    return {
        "answer": response['response'],
        "contexts": get_context(response),
    }

async def generate_ragas_dataset(agent, test_df):
    test_questions = test_df["query"].values
    responses = []
    
    for question in tqdm(test_questions):
        response = await generate_response(agent, question)  # Process each question serially
        responses.append(response)
    
    dataset_dict = {
        "question": test_questions,
        "answer": [response["answer"] for response in responses],
        "contexts": [response["contexts"] for response in responses],  # Store contexts separately for each question
    }
    ds = Dataset.from_dict(dataset_dict)
    return ds

# limit to first 5 rows for testing
with using_project("llama-index"):
    ragas_eval_dataset = await generate_ragas_dataset(agent, test_df.head(5))

ragas_evals_df = ragas_eval_dataset.to_pandas()
ragas_evals_df.head()

  0%|          | 0/5 [00:00<?, ?it/s]

Running step new_user_msg
Step new_user_msg produced event PrepEvent
Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input
Step handle_llm_input produced event ToolCallEvent
Running step handle_tool_calls
Step handle_tool_calls produced event PrepEvent
Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input
Step handle_llm_input produced event StopEvent
Running step new_user_msg
Step new_user_msg produced event PrepEvent
Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input
Step handle_llm_input produced event ToolCallEvent
Running step handle_tool_calls
Step handle_tool_calls produced event PrepEvent
Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input
Step handle_llm_input produced event ToolCallEvent
Running step handle_tool_calls
Step hand

question  \
0                                                 Who is the president?   
1  Who scored the most goals in the European Champion's League in 2020?   
2                      articles similar to the article on 'Philosophy'?   
3                                                  the school of athens   
4                                                              css flex   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 answer  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      The current president of the United States is Joe Biden, who assumed office on January 20, 2021.   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                  The top goalscorer in the UEFA Champions League for the 2020–21 season was Erling Haaland of Borussia Dortmund, who scored 10 goals.   
2                                                                                                                                                                                                                                                                                       Here are some articles similar to the article on 'Philosophy':\n\n1. [Western philosophy](https://en.wikipedia.org/wiki/Western_philosophy)\n2. [Contemporary philosophy](https://en.wikipedia.org/wiki/Contemporary_philosophy)\n3. [Doctor of Philosophy](https://en.wikipedia.org/wiki/Doctor_of_Philosophy)   
3  "The School of Athens" is a fresco by the Italian Renaissance artist Raphael. It was painted between 1509 and 1511 as part of a commission by Pope Julius II to decorate the rooms now known as the Stanze di Raffaello in the Apostolic Palace in Vatican City. The fresco depicts a congregation of ancient philosophers, mathematicians, and scientists, with Plato and Aristotle featured prominently in the center. The painting is notable for its use of accurate perspective projection and is regarded as one of Raphael's masterpieces, embodying the classical spirit of the Renaissance.   
4                                                                                                                                                                                                                                                                                                                                       Here are some articles similar to the article on 'CSS Flex':\n\n1. [CSS Flexible Box Layout](https://en.wikipedia.org/wiki/CSS_Flexible_Box_Layout)\n2. [CSS hack](https://en.wikipedia.org/wiki/CSS_hack)\n3. [NetSurf](https://en.wikipedia.org/wiki/NetSurf)   

                                                                                         

In [12]:
from phoenix.session.evaluation import get_qa_with_reference

# dataset containing span data for evaluation with Ragas
spans_dataframe = get_qa_with_reference(client, project_name="llama-index")
spans_dataframe.head(2)

No retrieval documents found.


AttributeError: 'NoneType' object has no attribute 'head'